<a href="https://colab.research.google.com/github/adimemes/Analysisi-Sentimen/blob/main/Analysis_Sentimen_Mobile_Legend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis Sentimen Mobile Legend

## Instal Package Libary Sastrawi dan Google play scaper


*   ```Sastrawi``` membantu menyederhanakan kata-kata yang mengalami infleksi dalam Bahasa Indonesia ke bentuk dasarnya
*   ```Google play scaper``` Membantu untuk Scraping data di Google play store



In [4]:
!pip install sastrawi
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00


## Import Libary yang dibutuhkan

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import string
import nltk
import csv
import requests
from io import StringIO
from google_play_scraper import app, reviews_all, Sort
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

pd.options.mode.chained_assignment = None
seed = 0
np.random.seed(seed)
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Scraping Data Di Playstore

Tahapan ini adalah tahapan untuk scraping data pada aplikasi Google Playstore, aplikasi yang diambil adalah Game Mobile Legend dengan rincian.


*   ```com.mobile.legends``` id aplikasi mobile legend dalam playstore
*   ```lang='id'``` Mengambil Review yang menggunakan bahasa indonesia
*   ```country='id'``` Mengambil review di negara indonesia
*   ```sort=Sort.MOST_RELEVANT``` Membuat data sesuai Urutan ulasan
*   ```count=4000``` Mengambil data sebanyak 4000 data

Kemudian setelah mendapatkan data tersebut lalu menyimpannya kedalam data yang berbentuk csv dengan header colomnya yaitu ```Review```



In [9]:
scrapreview = reviews_all(
    'com.mobile.legends',
    lang='id',             # 'id' mengambil review yang berbahasa indonsesia
    country='id',          # 'id' mengambil yang negara indonesia
    sort=Sort.MOST_RELEVANT,
    count=4000
)

with open('Scraping_ML_Dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrapreview:
        writer.writerow([review['content']])

In [31]:
ML_review = pd.DataFrame(scrapreview)
ML_review.shape
ML_review.head()
ML_review.to_csv('/content/Scraping_ML_Dataset.csv', index=False)

# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_data, jumlah_kolom = app_reviews_df.shape

## EDA (Exploratory Data Analysis)

In [32]:
ML_review.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ea63a4a4-f9db-4df5-9ac8-dc2233074530,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Tolong pihak developer, match nya yg seimbang ...",2,1222,1.9.70.10634,2025-05-09 16:07:35,None,NaT,1.9.70.10634
1,829493a2-0ca9-4d4a-89f9-6a9184a1bd92,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,2 tahun ini para pemain mengeluhkan dark siste...,1,2234,1.9.70.10634,2025-05-18 17:46:59,None,NaT,1.9.70.10634
2,c6e8ce46-8f1e-4301-bb08-7b87e44e224a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Gimana sih ni ada BUG PERTANDINGAN TIDAK DITEM...,1,553,1.9.70.10634,2025-05-16 03:08:25,None,NaT,1.9.70.10634
3,e1b961db-6b3d-4eec-afd4-6108aa44c67a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"sesuaikan gameplay, jaringan padahal bagus, pe...",5,2932,1.9.70.10634,2025-05-16 02:55:52,None,NaT,1.9.70.10634
4,9f6b521f-afd3-4cf7-9a7f-d2a0acb6c43f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"event collab naruto kali ini bagus, beneran ng...",5,22,1.9.72.10636,2025-05-22 14:35:15,None,NaT,1.9.72.10636


Awal adalah melihat data yang ada di dataset, terlihat pada kolom ini banyak terdapat data yang memiliki nilai None, NaT. kemungkinan pada data ini banyak yang memiliki missing value

In [33]:
ML_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225000 entries, 0 to 224999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              225000 non-null  object        
 1   userName              225000 non-null  object        
 2   userImage             225000 non-null  object        
 3   content               225000 non-null  object        
 4   score                 225000 non-null  int64         
 5   thumbsUpCount         225000 non-null  int64         
 6   reviewCreatedVersion  177497 non-null  object        
 7   at                    225000 non-null  datetime64[ns]
 8   replyContent          9786 non-null    object        
 9   repliedAt             9786 non-null    datetime64[ns]
 10  appVersion            177497 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 18.9+ MB


Terlihat pada info data, adanya data yang jomplang pada setiap kolom dataset yang mengakibatkan bisa merusak Analysis nantinya

### Memeriksa Missing Value dari dataset

In [34]:
ML_review.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,47503
at,0
replyContent,215214
repliedAt,215214


Terlihat pada data ini banyak sekali data yang memiliki missing Value

### Cleaning Data

Tahap awal yang dilakukan adalah membuat Dataframe baru yaitu ```ML_review_clean``` untuk menyimpan dataset yang sudah dibersihkan pada tahap awal adalah menghapus data yang memiliki nilai missing value dengan code
```.dropna()```

In [35]:
ML_review_clean = ML_review.dropna()

In [36]:
# Menghapus baris duplikat dari DataFrame ML_review_clean
ML_review_clean = ML_review_clean.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame ML_review_clean setelah menghapus duplikat
Jumlah_clean_data, Jumlah_clean_kolom = ML_review_clean.shape

In [38]:
print(f"Jumlah baris dan kolom sebelum penghapusan duplikat: {jumlah_data}, {jumlah_kolom}")
print(f"Jumlah baris dan kolom setelah penghapusan duplikat: {Jumlah_clean_data}, {Jumlah_clean_kolom}")

Jumlah baris dan kolom sebelum penghapusan duplikat: 225000, 11
Jumlah baris dan kolom setelah penghapusan duplikat: 8056, 11


In [39]:
ML_review_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8056 entries, 5 to 224966
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              8056 non-null   object        
 1   userName              8056 non-null   object        
 2   userImage             8056 non-null   object        
 3   content               8056 non-null   object        
 4   score                 8056 non-null   int64         
 5   thumbsUpCount         8056 non-null   int64         
 6   reviewCreatedVersion  8056 non-null   object        
 7   at                    8056 non-null   datetime64[ns]
 8   replyContent          8056 non-null   object        
 9   repliedAt             8056 non-null   datetime64[ns]
 10  appVersion            8056 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 755.2+ KB


## Data Preprocessing Text

In [41]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','GK','yaa','gak','nya','na','sih','ku',"di","ga","dll","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [48]:
slangwords = {"@": "di","abis": "habis", "masi": "masih", "bgt": "banget", "maks": "maksimal", "noob": "pemain pemula","gg": "good game",
              "feed": "mati terus", "feeder": "pemain yang mati terus", "afk": "meninggalkan permainan", "troll": "bermain asal",
              "carry": "menggendong tim", "ngelag": "koneksi buruk", "lag": "koneksi lambat", "op": "terlalu kuat", "nerf": "dilemahkan",
              "buff": "diperkuat", "push": "menyerang", "rank": "peringkat", "ml": "mobile legends", "mlbb": "mobile legends",
              "auto win": "pasti menang", "auto lose": "pasti kalah", "bocil": "anak kecil", "toxic": "berkata kasar", "idiot": "bodoh",
              "fix": "pasti", "anjir": "astaga", "anjay": "wow", "bangsat": "kasar", "kampret": "kesal","gk": "tidak","ga": "tidak",
              "ngga": "tidak","nggak": "tidak","gak": "tidak","bngt": "banget","bener": "benar","parah": "buruk","udah": "sudah",
              "dah": "sudah","bikin": "membuat","goblok": "bodoh","ngerti": "mengerti","ngambek": "marah","cape": "lelah","capek": "lelah",
              "btw": "ngomong-ngomong", "kwkw": "tertawa","wkwk": "tertawa","wkwwk": "tertawa","lol": "tertawa","ihh": "heran","pake": "pakai",
              "maen": "main", "mainin": "mainkan","tim": "team","temen": "teman", "ngekill": "membunuh","kill": "bunuh",
              }


In [49]:
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [50]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
ML_review_clean['text_clean'] = ML_review_clean['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
ML_review_clean['text_casefoldingText'] = ML_review_clean['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
ML_review_clean['text_slangwords'] = ML_review_clean['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
ML_review_clean['text_tokenizingText'] = ML_review_clean['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
ML_review_clean['text_stopword'] = ML_review_clean['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
ML_review_clean['text_akhir'] = ML_review_clean['text_stopword'].apply(toSentence)

In [51]:
ML_review_clean

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
5,d90a8fdf-16b0-4788-863e-e3791553bced,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kok jaringannya gini sih pas mau ngekill tiba2...,1,384,1.9.70.10634,2025-05-18 06:38:34,"Halo Kak,\nMaaf atas masalah jaringan yang Kak...",2025-05-18 07:10:08,1.9.70.10634,kok jaringannya gini sih pas mau ngekill tiba ...,kok jaringannya gini sih pas mau ngekill tiba ...,kok jaringannya gini sih pas mau membunuh tiba...,"[kok, jaringannya, gini, sih, pas, mau, membun...","[jaringannya, gini, pas, membunuh, koneksi, bu...",jaringannya gini pas membunuh koneksi buruk ku...
20,57a0b586-2d3e-4891-8abf-d95d007f604f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game titid!!!, Suka hilang koneksi tiba2 padah...",1,48,1.9.70.10634,2025-05-21 00:29:19,"Halo Kak,\nMaaf atas masalah jaringan yang Kak...",2025-05-21 02:54:46,1.9.70.10634,Game titid Suka hilang koneksi tiba padahal pi...,game titid suka hilang koneksi tiba padahal pi...,game titid suka hilang koneksi tiba padahal pi...,"[game, titid, suka, hilang, koneksi, tiba, pad...","[game, titid, suka, hilang, koneksi, ping, ijo...",game titid suka hilang koneksi ping ijo lagnya...
42,262eb85f-b766-4c87-89b9-24041d92571c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Moonton nggak jelas banget,, gue susah2 nyari ...",1,89,1.9.70.10634,2025-05-16 21:21:37,"Halo Kak,\nKami berkomitmen untuk menciptakan ...",2024-11-29 03:43:31,1.9.70.10634,Moonton nggak jelas banget gue susah nyari men...,moonton nggak jelas banget gue susah nyari men...,moonton tidak jelas banget gue susah nyari men...,"[moonton, tidak, jelas, banget, gue, susah, ny...","[moonton, banget, gue, susah, nyari, menang, e...",moonton banget gue susah nyari menang eh team ...
45,94f13107-096a-4446-a0ab-12380721b207,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"kenapa makin kesini makin menjadi ini game,mas...",1,130,1.9.64.10601,2025-04-27 00:03:05,"Halo Kak,\nMaaf atas masalah jaringan yang Kak...",2025-04-27 02:31:06,1.9.64.10601,kenapa makin kesini makin menjadi ini gamemasa...,kenapa makin kesini makin menjadi ini gamemasa...,kenapa makin kesini makin menjadi ini gamemasa...,"[kenapa, makin, kesini, makin, menjadi, ini, g...","[kesini, gamemasa, sinyal, bagus, ngelegtrus, ...",kesini gamemasa sinyal bagus ngelegtrus seakan...
46,a07b3cdb-fb31-4092-8ae2-204087e54f92,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"sering dianggap afk karena beberapa hal,sepert...",1,78,1.9.64.10601,2025-05-05 01:58:51,"Halo Kak,\nKami mohon maaf atas pengalaman kur...",2025-05-05 02:13:54,1.9.64.10601,sering dianggap afk karena beberapa halseperti...,sering dianggap afk karena beberapa halseperti...,sering dianggap meninggalkan permainan karena ...,"[sering, dianggap, meninggalkan, permainan, ka...","[dianggap, meninggalkan, permainan, halseperti...",dianggap meninggalkan permainan halseperti jar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224665,1dcb5b98-e1de-4e42-84cf-ae3f7c12babd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,moontoon dari sbulan yang lalu sering x pin me...,1,0,1.3.60.3801,2019-03-29 08:41:28,"Dear Player,\n\nValir dan Tim sedang berusaha ...",2019-03-31 05:27:48,1.3.60.3801,moontoon dari sbulan yang lalu sering x pin me...,moontoon dari sbulan yang lalu sering x pin me...,moontoon dari sbulan yang lalu sering x pin me...,"[moontoon, dari, sbulan, yang, lalu, sering, x...","[moontoon, sbulan, x, pin, merah, pin, kuning,...",moontoon sbulan x pin merah pin kuning lagku h...
224674,54deebfc-f5ee-4890-993c-c45710e07bdd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Masalah jaringan,kenapa jaringan nya tidak sta...",2,14,1.7.94.8681,2023-07-12 17:49:54,"Dear Hero,\n

In [53]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [54]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [56]:
results = ML_review_clean['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
ML_review_clean['polarity_score'] = results[0]
ML_review_clean['polarity'] = results[1]
print(ML_review_clean['polarity'].value_counts())

polarity
negative    4905
positive    3151
Name: count, dtype: int64
